In [ ]:
##############################################################
########################## Notes #############################
# This file is to calculate the bypass factor.
# Use formula here to calculate: EnergyPlus Enginnering Reference - Coils
# https://bigladdersoftware.com/epx/docs/8-2/engineering-reference/coils.html
# Is Useful to check the results: https://drajmarsh.bitbucket.io/psychro-chart2d.html

In [5]:
# !pip3 install CoolProp

# not work when connect to PNNL VPN

                                              0.0/2.0 MB ? eta -:--:--
     ---                                      0.2/2.0 MB 5.3 MB/s eta 0:00:01
     -----------------                        0.8/2.0 MB 10.6 MB/s eta 0:00:01
     ------------------------------------     1.8/2.0 MB 19.1 MB/s eta 0:00:01
     ---------------------------------------- 2.0/2.0 MB 13.8 MB/s eta 0:00:00


In [10]:
# Yiting's scripts

from scipy.optimize import fsolve
import numpy as np



def calculate_w_leaving(T_entering, w_entering, T_leaving, Q_total, st, m_dot, c_p, h_fg):
    
    # Difference between the total heat removal and the sum of sensible and latent heat.
    def heat_balance(w_leaving):
        Q_sensible = m_dot * c_p * (T_entering - T_leaving)
        Q_latent = m_dot * w_entering * h_fg - m_dot * w_leaving * h_fg
        return Q_total * st - Q_latent
    
    # Using fsolve to find T_leaving where the heat balance is zero
    T_adp = fsolve(heat_balance, 0.082)[0]
    
    return T_adp


# Example
T_entering = 26.67    # Entering air dry bulb temperature in °C
w_entering = 0.0112    # Entering air humidity ratio in kg/kg
T_leaving = 9.93
st = 0.66
Q_total = 11488.3847155 / 39.2 * 41.7 / 1000   # Total heat removal in Watts
m_dot = 1.19 * 4011.55 / 1000     # Air mass flow rate in kg/s
c_p = 1.006           # Specific heat of air in kJ/kg.K
h_fg = 2260        # Enthalpy of vaporization in kJ/kg

adp = calculate_w_leaving(T_entering, w_entering, T_leaving, Q_total, st, m_dot, c_p, h_fg)
print(f"Leaving Evaporator Humidity Ratio: {adp:.2f} kg/kg")

## Make an initial estimate of the apparatus dew point temperature
def pw2dp(w, P):
    # Constants for the Magnus formula
    l = 243.7
    beta = 17.27
    Pw0 = 611.2  # Saturation pressure of water vapor at 0°C in Pa

    # Calculate the partial pressure of water vapor
    Pw = (w * P) / (0.622 + w)
    
    # Use the Magnus formula to compute the dew point
    # numerator = B * np.log(Pw/Pw0)
    # denominator = A - np.log(Pw/Pw0)
    # Td = numerator / denominator
    num = 243.12 * np.log(Pw/Pw0)
    den = 17.62 - np.log(Pw/Pw0)
    Td = num / den
    return Td


## Equation 1
def pdp2w(Td, P):
    # Constants for the Magnus formula
    A = 17.27
    B = 237.7
    Pw0 = 611.2  # Saturation pressure of water vapor at 0°C in Pa
    # Calculate the saturation pressure at the dew point
    Pws = Pw0 * np.exp((A * Td) / (B + Td))    
    # The partial pressure of water vapor is equal to the saturation pressure at dew point
    Pw = Pws    
    # Calculate the humidity ratio
    w = (0.622 * Pw) / (P - Pw)
    return w


## The “slope” of temperature versus humidity ratio on the psych chart 
## betweween the inlet and outlet air conditions
def st_slope(T_in, T_out, w_in, w_out):
    st = (T_in - T_out) / (w_in - w_out)
    return st


## Iterating
def ADP(T_in, w_in, Tad0, st, epsilon = 0.01):
    Pa = 101325
    Tad = Tad0 # make an initial estimate of the apparatus dew point temperature
    for i in range(1000):
        ## Equation 2
        w = pdp2w(Tad, Pa)
        Tad_new = T_in - st*(w_in - w)
        print("w,Tad_new")
        print(w,Tad_new)
        if np.abs(Tad_new - Tad) <= epsilon:
            return Tad_new, w
        Tad = Tad_new
    return None


## The apparatus dewpoint enthalpy is then:
def compute_enthalpy(T, w):
    # Constants
    cp = 1.006  # Specific heat of dry air at constant pressure (kJ/kg·°C)
    hv = 2501   # Enthalpy of vaporization of water at 0°C (kJ/kg)
    # Compute the enthalpy
    h = cp * T + w * (hv + 1.86*T)
    return h




Leaving Evaporator Humidity Ratio: 0.01 kg/kg


In [11]:
# Yiting's Test

T_in = 26.67
w_in = 0.0112
T_out = 9.93
w_out = 0.01
Pa = 101325
st = st_slope(T_in, T_out, w_in, w_out)
print("s/t:", st)
Tad0 = pw2dp(w_out, Pa)  # Initial guess of ADP
print("ADP init:", Tad0, "°C")
print("ADP:", ADP(T_in, w_in, Tad0, st), "°C")
T_adp, w_adp = ADP(T_in, w_in, Tad0, st)
h_adp = compute_enthalpy(T_adp, w_adp)
h_in = compute_enthalpy(T_in, w_in)
h_out = compute_enthalpy(T_out, w_out)


## The coil bypass factor is:
BF = (h_out - h_adp) / (h_in - h_adp)
print("BF:", BF)


s/t: 13950.000000000005
ADP init: 14.07662393598085 °C
0.0100121148353774 10.099001953514726
0.007678162341030634 -22.459635342622676
0.0006195281186943684 -120.92758274421361
6.41366223336722e-11 -129.5699991052942
3.862246656436672e-12 -129.5699999461217
ADP: (-129.5699999461217, 3.862246656436672e-12) °C
0.0100121148353774 10.099001953514726
0.007678162341030634 -22.459635342622676
0.0006195281186943684 -120.92758274421361
6.41366223336722e-11 -129.5699991052942
3.862246656436672e-12 -129.5699999461217
BF: 0.8928571428201955


In [18]:
## Jeremy's scripts

from CoolProp.HumidAirProp import HAPropsSI

def find_apparatus_dew_point(w_in, w_out, t_in, t_out):
    # slpoe
    a = (w_out - w_in) / (t_out - t_in)
    # w_x = a * t_x + b
    b = w_in - a * t_in
    t_adp = t_out # Equation 2
    incr = 0.001
    t_x = t_out + incr
    
    found = True
    while found:
        t_x += incr
        w_x = a * t_x + b
        t_adp = HAPropsSI('D','T',t_x + 273.15,'P',101325,'W', w_x) - 273.15
        if abs(t_adp - t_x) < 0.001:
            break
        incr = (t_adp - t_x) / 10
    return t_adp

## Test on this:
w_in, w_out, T_in, T_out = 0.01112, 0.01, 23, 13
T_adp = round(find_apparatus_dew_point(w_in, w_out, T_in, T_out), 2)
print("T_adp:", T_adp)

Pa = 101325
W_in = pdp2w(T_in, Pa)
W_out = pdp2w(T_out, Pa)
w_adp = pdp2w(T_adp, Pa)
h_adp = compute_enthalpy(T_adp, w_adp)
h_in = compute_enthalpy(T_in, w_in)
h_out = compute_enthalpy(T_out, w_out)
BF_byT = (T_out - T_adp) / (T_in - T_adp)
BF_byH = (h_out - h_adp) / (h_in - h_adp)
print("BF_byT:", BF_byT)
print("BF_byH:", BF_byH)
print(W_in,W_out)

T_adp: 14.18
BF_byT: -0.13378684807256233
BF_byH: -0.12100054366688948
0.017707267627385063 0.009324906442953121


In [ ]:
## revised script

def bypass_factor(w_in, w_out, t_in, t_out):
    # slpoe
    a = (w_out - w_in) / (t_out - t_in)
    # w_x = a * t_x + b
    b = w_in - a * t_in
    t_adp = t_out # Equation 2
    incr = 0.001
    t_x = t_out + incr
    
    found = True
    while found:
        t_x += incr
        w_x = a * t_x + b
        t_adp = HAPropsSI('D','T',t_x + 273.15,'P',101325,'W', w_x) - 273.15
        if abs(t_adp - t_x) < 0.001:
            break
        incr = (t_adp - t_x) / 10
    return t_adp

In [17]:
import math

def calculate_w(tdb, twb): # humidity_ratio
    vapor_pressure = 6.112 * math.exp((17.67 * twb) / (twb + 243.5))
    humidity_ratio = 0.622 * (vapor_pressure / (101.325 - vapor_pressure))
    return humidity_ratio

def calculate_tldb(tedb, sc, cfm):
    tldb = tedb - sc / (1.1 * cfm)
    return tldb

def calculate_hlwb(hewb, tc, cfm):
    hlwb = hewb - tc / (4.5 * cfm)
    return hlwb
    
def compute_h(T, w):
    cp = 1.006  # Specific heat of dry air at constant pressure (kJ/kg·°C)
    hv = 2501   # Enthalpy of vaporization of water at 0°C (kJ/kg)
    h = cp * T + w * (hv + 1.86*T)
    return h

def compute_bf_T(tldb,tedb,tadp):
    BF_byT = (tldb - tadp) / (tedb - tadp)
    return BF_byT

def compute_bf_H(h_out,h_in,h_adp):
    BF_byH = (h_out - h_adp) / (h_in - h_adp)
    return BF_byH




Humidity Ratio: 0.186463


In [ ]:
# Example usage

dry_bulb_temperature = 25.0  # in degrees Celsius, 77F
wet_bulb_temperature = 20.0  # in degrees Celsius, 68F

humidity_ratio = calculate_w(dry_bulb_temperature, wet_bulb_temperature)
print(f'Humidity Ratio: {humidity_ratio:.6f}')